In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split;
from sklearn import preprocessing
import matplotlib.pyplot as plt
import scipy.io as sio
import h5py
import psutil
import os

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!ls
!mkdir -p drive
!google-drive-ocamlfuse drive
import os
os.chdir("drive/FinalKaggle")

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.
datalab


In [0]:
file = h5py.File('train_data.mat','r')
x = file['train_feat'][:]
y = file['train_label'][:]
x = x.T
y = y.T

In [3]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8)
x_train.shape

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


(60992, 6812)

In [0]:
import torch.utils.data as Data
import torch as t
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

tensor_data = t.from_numpy(x)

ids = t.LongTensor(y).view(-1,1)
tensor_target = t.zeros(76240, 397).scatter_(1, ids, 1)
tensor_target = tensor_target.long()
targetnp=tensor_target.numpy()
idxs=np.where(targetnp>0)[1]
tensor_target=t.LongTensor(idxs)

torch_dataset = Data.TensorDataset(data_tensor=tensor_data, target_tensor=tensor_target)
loader = Data.DataLoader(dataset=torch_dataset, batch_size=256, shuffle=True)

In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
import torch.utils.data as Data
import torch as t
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

class Net(nn.Module):
    def __init__(self):

        super(Net,self).__init__()
        
        self.fc_bn = nn.BatchNorm2d(6812)
        self.fc1 = nn.Linear(6812, 4500)
        self.dp1 = nn.Dropout(0.25)
        self.fc_bn1 = nn.BatchNorm2d(4500)
        self.fc2 = nn.Linear(4500, 4500)
        self.dp2 = nn.Dropout(0.25)
        self.fc_bn2 = nn.BatchNorm2d(4500)
        self.fc3 = nn.Linear(4500, 397)
 
    def forward(self,x):
        out = self.fc_bn(x)
        out = self.fc_bn1(F.relu(self.dp2(self.fc1(out))))
        out = self.fc_bn2(F.relu(self.dp2(self.fc2(out))))
        out = self.fc3(out)
        return out
 

if __name__ == "__main__":
 
    net = Net()



In [17]:
print(net)
params = list(net.parameters())
print(len(params))
for name, parameters in net.named_parameters():
  print(name, "：", parameters.size())

Net(
  (fc_bn): BatchNorm2d(6812, eps=1e-05, momentum=0.1, affine=True)
  (fc1): Linear(in_features=6812, out_features=4500)
  (dp1): Dropout(p=0.25)
  (fc_bn1): BatchNorm2d(4500, eps=1e-05, momentum=0.1, affine=True)
  (fc2): Linear(in_features=4500, out_features=4500)
  (dp2): Dropout(p=0.25)
  (fc_bn2): BatchNorm2d(4500, eps=1e-05, momentum=0.1, affine=True)
  (fc3): Linear(in_features=4500, out_features=397)
)
12
fc_bn.weight ： torch.Size([6812])
fc_bn.bias ： torch.Size([6812])
fc1.weight ： torch.Size([4500, 6812])
fc1.bias ： torch.Size([4500])
fc_bn1.weight ： torch.Size([4500])
fc_bn1.bias ： torch.Size([4500])
fc2.weight ： torch.Size([4500, 4500])
fc2.bias ： torch.Size([4500])
fc_bn2.weight ： torch.Size([4500])
fc_bn2.bias ： torch.Size([4500])
fc3.weight ： torch.Size([397, 4500])
fc3.bias ： torch.Size([397])


In [0]:
from sklearn.metrics import accuracy_score
import torch.backends.cudnn as cudnn
use_cuda = t.cuda.is_available()

if use_cuda == True:
    net.cuda()
    # multi gpu setting
    net = t.nn.DataParallel(
        net, device_ids=range(t.cuda.device_count()))
    cudnn.benchmark = True
net.load_state_dict(t.load('net_4096.pwf'))

KeyError: ignored

In [8]:
from sklearn.metrics import accuracy_score
import torch.backends.cudnn as cudnn
use_cuda = t.cuda.is_available()

if use_cuda == True:
    net.cuda()
    # multi gpu setting
    net = t.nn.DataParallel(
        net, device_ids=range(t.cuda.device_count()))
    cudnn.benchmark = True

for epoch in range(30):
    loss_fn = nn.CrossEntropyLoss()
    for step, (batch_x, batch_y) in enumerate(loader):
        net.train()
        if use_cuda:
            batch_x, batch_y = batch_x.cuda(), batch_y.cuda()
        input_ = Variable(batch_x)
        target = Variable(batch_y.long())
        optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-5)
        optimizer.zero_grad()
        output = net(input_)
        loss =  loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if step % 400 == 0:
          net.eval()
          _, predicted = t.max(output.data, 1)
          q = predicted.cpu().numpy()
          p = target.data.cpu().numpy()
          print('Accuracy : {}'.format(accuracy_score(p, q)))
          print('Epoch : {}'.format(epoch))
          print('Step : {}'.format(step))
          print('Loss: {:.6f}'.format(loss.data[0]))

Accuracy : 1.0
Epoch : 0
Step : 0
Loss: 0.104171


KeyboardInterrupt: ignored

In [0]:
file = h5py.File('test_data_raw.mat','r')
t_x = file['test_feat'][:]
t_x = t_x.T

In [0]:
net.eval()
inputs = Variable(t.from_numpy(t_x))
outputs = net(inputs)
_, predicted = t.max(outputs.data, 1)

In [11]:
sample = pd.read_csv('submission_sample.csv', header=None)
sample_id = sample[0]
sample_id[0]

'image'

In [0]:
p = predicted.cpu().numpy()
import csv
with open("final.csv","w") as csvfile: 
    writer = csv.writer(csvfile, lineterminator='\n')
    writer.writerow(["image","label"])
    for i in range(0,19850):
        writer.writerow([sample_id[i + 1], str(p[i])])

In [12]:
inputs = Variable(t.from_numpy(x_test))
outputs = net(inputs)
_, pr = t.max(outputs.data, 1)
q = pr.cpu().numpy()
from sklearn.metrics import accuracy_score
s1 = accuracy_score(y_test, q)
print(s1)

0.3854931794333683


In [0]:
inputs = Variable(t.from_numpy(x_train))
outputs = net(inputs)
_, pr = t.max(outputs.data, 1)
q = pr.cpu().numpy()
from sklearn.metrics import accuracy_score
s2 = accuracy_score(y_train, q)
print(s2)

In [0]:
t.save(net.state_dict(), 'net_30_30.pwf')

In [0]:
inputs = Variable(t.from_numpy(x))
outputs = net(inputs)
_, pr = t.max(outputs.data, 1)
q = pr.cpu().numpy()
from sklearn.metrics import accuracy_score
s3 = accuracy_score(y, q)
print(s3)

RuntimeError: ignored